True

In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.is_available()
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [ ]:
from ultralytics import YOLO
import os

#os.environ['TORCH_USE_CUDA_DSA'] = "1"
# Load a model
model = YOLO("yolov8n-seg.yaml").load("yolov8n.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="yolo/dataset.yaml", 
                      epochs=300, 
                      imgsz=512,
                      batch=10, 
                      save=True,
                      device = 0, 
                      project='train_yolo',
                      pretrained = 'yolov8n.pt',
                      optimizer = 'auto',
                      verbose = True, 
                      val = True, 
                      half = True,
                      cache = 'disk'
                      )

In [7]:
from ultralytics.utils.plotting import plot_results

plot_results('./train_yolo/train3/results.csv', segment=True)

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
df = pd.read_csv('./train_yolo/train3/results.csv')

df.head()
df.columns = pd.Series(df.columns).apply(lambda x : str(x).strip())

df.columns

,epoch,train/box_loss,train/seg_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),metrics/mAP50-95(M),val/box_loss,val/seg_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,0.43686,1.36780,1.31570,0.84349,0.98479,0.98562,0.99402,0.95039,0.98513,0.98533,0.99404,0.92319,0.30123,0.39844,0.46448,0.79209,0.003331,0.003331,0.003331
1,2,0.42143,0.47336,0.62845,0.84044,0.97978,0.98008,0.99379,0.95459,0.97986,0.98016,0.99379,0.91789,0.28346,0.34522,0.34622,0.80024,0.006658,0.006658,0.006658
2,3,0.44348,0.44661,0.54885,0.85143,0.98722,0.99153,0.99376,0.95624,0.98785,0.99091,0.99377,0.92741,0.28533,0.34056,0.29987,0.80188,0.009978,0.009978,0.009978
3,4,0.43023,0.43325,0.48632,0.85104,0.97049,0.98380,0.99078,0.95686,0.97049,0.98380,0.99088,0.92949,0.27344,0.32117,0.27212,0.79818,0.009970,0.009970,0.009970
4,5,0.39779,0.41314,0.41247,0.84271,0.97572,0.97339,0.99297,0.96484,0.97572,0.97339,0.99297,0.92801,0.27276,0.33615,0.26912,0.79692,0.009960,0.009960,0.009960


In [25]:
savedir = 'train_yolo/plot_results/'

if not os.path.exists(savedir):
    os.mkdir(savedir)

for column in df.columns : 
    if column == 'epoch':
        continue
    # Example of plotting the 'loss' column
    plt.figure(figsize=(10, 6))
    plt.plot(df['epoch'], df[column], marker='o', linestyle='-', color='b')
    plt.title(column+' per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel(column)
    plt.grid(True)
    plt.savefig(fname = savedir+column.replace('/', '')+'.jpg')
    plt.close()

In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO("train_yolo/train3/weights/best.pt")

In [77]:
# Validate the model
metrics = model.val(data ='yolo/dataset.yaml', 
                    batch = 32,
                    plots = True,
                    split = 'test',
                    imgsz = 512,
                    save_json = True)  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps  # a list contains map50-95(B) of each category
metrics.seg.map  # map50-95(M)
metrics.seg.map50  # map50(M)
metrics.seg.map75  # map75(M)
metrics.seg.maps  # a list contains map50-95(M) of each category

Ultralytics YOLOv8.2.8 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)


val: Scanning /home/yw/code/PET/yolo/labels/test.cache... 1991 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1991/1991 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:46<00:00,  1.36it/s]


                   all       1991       5863          1      0.999      0.995      0.993          1      0.999      0.995      0.984
                   PET       1991       1504          1      0.998      0.995      0.991          1      0.998      0.995      0.975
                    PS       1991       1574          1          1      0.995      0.994          1          1      0.995      0.978
                    PP       1991       1376      0.999      0.999      0.995      0.995          1      0.999      0.995      0.994
                    PE       1991       1409          1          1      0.995      0.995          1          1      0.995      0.988
Speed: 0.4ms preprocess, 2.6ms inference, 0.0ms loss, 1.3ms postprocess per image
Saving runs/segment/val3/predictions.json...
Results saved to runs/segment/val3


array([    0.97485,     0.97811,     0.99433,     0.98843])

In [78]:
print(metrics.box.map)  # map50-95(B)
print(metrics.box.map50)  # map50(B)
print(metrics.box.map75)  # map75(B)
metrics.box.maps  # a list contains map50-95(B) of each category

0.9933852553536873
0.995
0.9949686062142626


array([    0.99055,     0.99356,     0.99475,     0.99468])

In [ ]:
'''p (list): Precision for each class. Shape: (nc,).
        r (list): Recall for each class. Shape: (nc,).
        f1 (list): F1 score for each class. Shape: (nc,).
        all_ap (list): AP scores for all classes and all IoU thresholds. Shape: (nc, 10).
        ap_class_index (list): Index of class for each AP score. Shape: (nc,).
        nc (int): Number of classes.

    Methods:
        ap50(): AP at IoU threshold of 0.5 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
        ap(): AP at IoU thresholds from 0.5 to 0.95 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
        mp(): Mean precision of all classes. Returns: Float.
        mr(): Mean recall of all classes. Returns: Float.
        map50(): Mean AP at IoU threshold of 0.5 for all classes. Returns: Float.
        map75(): Mean AP at IoU threshold of 0.75 for all classes. Returns: Float.
        map(): Mean AP at IoU thresholds from 0.5 to 0.95 for all classes. Returns: Float.
        mean_results(): Mean of results, returns mp, mr, map50, map.
        class_result(i): Class-aware result, returns p[i], r[i], ap50[i], ap[i].
        maps(): mAP of each class. Returns: Array of mAP scores, shape: (nc,).
        fitness(): Model fitness as a weighted combination of metrics. Returns: Float.
        update(results): Update metric attributes w'''

In [79]:
metrics.seg.mean_results()

[0.9999283123945675, 0.9993196437407225, 0.995, 0.9839296852782284]

# For adjusting the font size in the graph, I modified the original plotting function

In [50]:
import numpy as np               # For numerical operations
import matplotlib.pyplot as plt  # For plotting
import seaborn as sns            # For creating the heatmap
import warnings                  # For handling warnings
from pathlib import Path         # For handling file paths

def plot(self, normalize=True, save_dir="", names=(), on_plot=None):
    """
    Plot the confusion matrix using seaborn and save it to a file.

    Args:
        normalize (bool): Whether to normalize the confusion matrix.
        save_dir (str): Directory where the plot will be saved.
        names (tuple): Names of classes, used as labels on the plot.
        on_plot (func): An optional callback to pass plots path and data when they are rendered.
    """
    import seaborn  # scope for faster 'import ultralytics'

    array = self.matrix / ((self.matrix.sum(0).reshape(1, -1) + 1e-9) if normalize else 1)  # normalize columns
    array[array < 0.005] = np.nan  # don't annotate (would appear as 0.00)

    fig, ax = plt.subplots(1, 1, figsize=(12, 9), tight_layout=True)
    nc, nn = self.nc, len(names)  # number of classes, names
    seaborn.set_theme(font_scale=1 if nc < 50 else 0.8)  # for label size
    labels = (0 < nn < 99) and (nn == nc)  # apply names to ticklabels
    ticklabels = (list(names) + ["background"]) if labels else "auto"
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # suppress empty matrix RuntimeWarning: All-NaN slice encountered
        heatmap = seaborn.heatmap(
            array,
            ax=ax,
            annot=nc < 30,
            annot_kws={"size": 23},
            cmap="Blues",
            fmt=".2f" if normalize else ".0f",
            square=True,
            vmin=0.0,
            xticklabels=ticklabels,
            yticklabels=ticklabels,
            linecolor='black', 
            linewidths=1
        )
        heatmap.set_facecolor((1, 1, 1))
        
        cbar = heatmap.collections[0].colorbar
        cbar.ax.tick_params(labelsize=18)
    title = "Confusion Matrix" + " Normalized" * normalize
    ax.set_xlabel("True", fontsize=23)
    ax.set_ylabel("Predicted", fontsize=23)
    ax.set_title(title, fontsize=25)

    
    # Set tick parameters for both x and y axes
    ax.tick_params(axis='both', which='major', labelsize=20)  # Adjust the labelsize as needed

    plot_fname = Path(save_dir) / f'{title.lower().replace(" ", "_")}.png'
    fig.savefig(plot_fname, dpi=400)
    plt.close(fig)
    if on_plot:
        on_plot(plot_fname)


In [81]:
plot(self = metrics.confusion_matrix, normalize=False, save_dir='',names = ['PET', 'PS', 'PP', "PE"], on_plot=None)

In [116]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def plot_metrics(metrics, save_dir):
    curves_results = metrics.curves_results
    curves_names = metrics.curves

    for i, curve_name in enumerate(curves_names):

        fs = 18
        
        x_data = curves_results[i][0]
        y_data = np.array(curves_results[i][1])
        
        # Plotting each category
        fig, ax = plt.subplots(figsize=(7, 6))
        for j, label in enumerate(['PET', 'PS', 'PP', 'PE']):
            ax.plot(x_data, y_data[j], label=label)
        
        # Calculate and plot the average curve
        y_avg = np.mean(y_data, axis=0)
        lbltxt = f"AVG : all classes {y_avg.max():.2f} at {x_data[y_avg.argmax()]:.3f}"
        if ("-recall" in curve_name.lower()):
            lbltxt = f"AVG : all classes 0.995 @mAP50"
        elif ("recall-confi" in curve_name.lower()):
            mask = y_avg < 0.9999
            if np.any(mask):  # Check if there is any value less than 0.9999
                min_index = np.argmin(y_avg[mask])
                lbltxt = f"AVG : all classes {y_avg.max():.2f} at {x_data[min_index]:.3f}"
        ax.plot(x_data, y_avg, linewidth=1, color="blue", label=lbltxt)
        
        # Adding titles and labels
        curve_name = curve_name.replace('(B)', '-Box')
        curve_name = curve_name.replace('(M)', '-Mask')

        ax.set_title(curve_name, fontsize=fs)
        ax.set_xlabel(curve_name.split('-')[1], fontsize=fs)
        ax.set_ylabel(curve_name.split('-')[0], fontsize=fs)
        
        fs2 = 13
        ax.legend(fontsize=fs2)
        
        # Adjusting tick parameters
        ax.tick_params(axis='both', which='major', labelsize=fs2, colors='black')
        ax.tick_params(axis='both', which='minor', labelsize=fs2, colors='black')

        # Change the color of the spines
        for spine in ax.spines.values():
            spine.set_color('black')
        
        # Change the color of the grid lines
        ax.grid(True, color='black', linestyle='--', linewidth=0.5)
        
        # Save the figure
        save_path = Path(save_dir) / f'{curve_name.replace(" ", "_")}.png'
        fig.savefig(save_path, dpi=400, bbox_inches='tight')
        plt.close(fig)

# Example usage
save_directory = 'testgraph'
plot_metrics(metrics, save_directory)


In [8]:
#benchmark on test set

from ultralytics.utils.benchmarks import benchmark

import os 

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Benchmark on GPU
benchmark(model="train_yolo/train3/weights/best.pt", data="dataset_t.yaml", imgsz=512, half=False, device=1)

Setup complete ✅ (24 CPUs, 93.1 GB RAM, 2271.2/3724.2 GB disk)

Benchmarks complete for best.pt on dataset_t.yaml at imgsz=512 (45.51s)
                   Format Status❔  Size (MB)  metrics/mAP50-95(M)  Inference time (ms/im)
0                 PyTorch       ✅       12.8               0.9743                    6.56
1             TorchScript       ✅       12.9               0.9725                    2.95
2                    ONNX       ❌        0.0                  NaN                     NaN
3                OpenVINO       ❌        0.0                  NaN                     NaN
4                TensorRT       ❌        0.0                  NaN                     NaN
5                  CoreML       ❌        0.0                  NaN                     NaN
6   TensorFlow SavedModel       ❌        0.0                  NaN                     NaN
7     TensorFlow GraphDef       ❌        0.0                  NaN                     NaN
8         TensorFlow Lite       ❌        0.0          

,Format,Status❔,Size (MB),metrics/mAP50-95(M),Inference time (ms/im)
0,PyTorch,✅,12.8,0.9743,6.56
1,TorchScript,✅,12.9,0.9725,2.95
2,ONNX,❌,0.0,NaN,NaN
3,OpenVINO,❌,0.0,NaN,NaN
4,TensorRT,❌,0.0,NaN,NaN
5,CoreML,❌,0.0,NaN,NaN
6,TensorFlow SavedModel,❌,0.0,NaN,NaN
7,TensorFlow GraphDef,❌,0.0,NaN,NaN
8,TensorFlow Lite,❌,0.0,NaN,NaN
9,TensorFlow Edge TPU,❌,0.0,NaN,NaN


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load an official model
model = YOLO("path/to/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

In [ ]:
# check if segmentation region is correct.
import cv2
import numpy as np
import matplotlib.pyplot as plt

def read_yolo_txt(txt_file):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
    masks = []
    for line in lines:
        parts = line.strip().split()
        cls = int(parts[0])
        coords = list(map(float, parts[1:]))
        masks.append((cls, coords))
    return masks

def draw_masks_on_image(image, masks):
    h, w, _ = image.shape
    mask_image = image.copy()

    for cls, coords in masks:
        # Convert normalized coordinates to pixel coordinates
        points = np.array(coords).reshape(-1, 2)
        points[:, 0] *= w
        points[:, 1] *= h
        points = points.astype(np.int32)

        # Draw the polygon on the image
        cv2.polylines(mask_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.fillPoly(mask_image, [points], color=(0, 255, 0, 128))

    return mask_image

def visualize_image_with_masks(image_path, txt_path):
    image = cv2.imread(image_path)
    masks = read_yolo_txt(txt_path)
    mask_image = draw_masks_on_image(image, masks)

    # Convert BGR to RGB for displaying with matplotlib
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(mask_image)
    plt.axis('off')
    plt.show()

# Example usage
image_path = 'yolo/images/train/PE_068_36.jpg'
txt_path = 'yolo/labels/train/PE_068_36.txt'
visualize_image_with_masks(image_path, txt_path)


In [ ]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
import cv2

# Load model
model = YOLO("yolo_traintst/train5/weights/best.pt")

# Make predictions
results = model.predict(source='yolo/images/val/PE_068_1005.jpg', device=0)
def visualize_image_with_masks(image_path, masks):
    image = cv2.imread(image_path)
    masks = masks
    mask_image = draw_masks_on_image(image, masks)

    # Convert BGR to RGB for displaying with matplotlib
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(mask_image)
    plt.axis('off')
    plt.show()

for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")

    #print(probs)
    #visualize_image_with_masks ('yolo/images/val/PE_068_1005.jpg',  masks=masks)

    